In [2]:
from langchain_neo4j import Neo4jGraph

In [5]:
graph = Neo4jGraph(
                url="bolt://localhost:7687",
                username="neo4j",
                password="neo4jpassword",
                refresh_schema=False
            )

graph

In [3]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_ollama import ChatOllama
from langchain_core.documents import Document

llm = ChatOllama(model="gemma3:1b",
    temperature=0)
llm_transformer = LLMGraphTransformer(llm=llm)

In [17]:
text = """
 Aker BP is committed to reducing CO2 emissions corresponding to our share of the KonKraft obligations in support of the Paris climate agreement. 
 All our operations are located in Norway and 
have scope 1 and scope 2 emissions. Scope 3 emissions are partly estimated. Direct greenhouse gas (GHG) emissions (scope 1) 
were 891350 tonnes CO2e in 2018 compared to 913796 tonnes CO2e in 2017. Indirect GHG emissions (scope 2) increased from 
126180 tonnes CO2e in 2017 to 159391 tonnes CO2e in 2018, 
mainly due to increased sea water injection. GHG emissions 
include carbon dioxide (CO2) and methane (CH4). Emissions of 
NOx and SOX
 increased with 13 % and 11 % respectively, mainly 
due to increased drilling activity in 2018
"""

documents = [Document(page_content=text)]

graph_documents = await llm_transformer.aconvert_to_graph_documents(documents)

In [25]:
allowed_nodes = ['Company', 'Report', 'Section', 'Metric', 'Measure', 'Target', 'Assurance', 'Claim', 'Evidence', 'Standard', 'Location', 'Methodology']

allowed_relationships = [
    ('Company', 'PUBLISHED', 'Report'),
    ('Report', 'HAS_SECTION', 'Section'),
    ('Section', 'MENTIONS', 'Metric'),
    ('Metric', 'MEASURED_AS', 'Measure'),
    ('Measure', 'COVERING', 'Location'),
    ('Measure', 'SUPPORTED_BY', 'Evidence'),
    ('Target', 'COVERS', 'Metric'),
    ('Report', 'ASSURED_BY', 'Assurance'),
]

graph_transformer_nodes_defined = LLMGraphTransformer(llm=llm, allowed_nodes=allowed_nodes, allowed_relationships=allowed_relationships, 
                                                      additional_instructions="You are an expert in the field of ESG. Your task is to extract ESG disclosures and identify them as nodes and establish relationships between these nodes.")
graph_documents_nodes_defined = await graph_transformer_nodes_defined.aconvert_to_graph_documents(documents)



In [28]:
graph_documents_nodes_defined[0].relationships

[]